In [1]:
import numpy as np
import textblob as textblob
import praw
from datetime import datetime
import matplotlib.pyplot as plt
import json
import creds
from pprint import pprint
import pandas as pd
from time import time

In [2]:
#Importing NLTK library and associated packaged

import nltk
nltk.__version__
# nltk.download('vader_lexicon')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag_sents
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Importing textblob to compare sentiment analysis results with those from nltk

from textblob import TextBlob

# Functions

In [3]:
#pulls all discussion ids available and adds

def get_disc_ids(subreddit, disc_dict):
    
    discussion_ids = []
    for submission in subreddit.search('Daily Discussion Post'):
        if 'Daily Discussion Post' in submission.title:
            discussion_ids.append(submission.id)
      
    for d in discussion_ids:
        if d not in disc_dict.keys():
            print("Added", d, "to discussion dictionary.")
            disc_dict[d] = {}
        else: 
            pass
    
    if len(discussion_ids) != len(disc_dict):
        print("Alert! Length of discussion_ids does not equal length of dictionary.")
    
    return disc_dict

In [4]:
def comment_info(us_com, submission, sid):
    comment = us_com.body
    comment = comment.replace('\n', ' ')
    comment = comment.replace('I\'m', 'i am').replace('i\'m', 'i am').replace('i\'ll', 'i will').replace('I\'ll', 'i will')
    comment = comment.lower()
    
    #grab date/ time info for each comment 
    utc = submission.created_utc
    dt_object = datetime.fromtimestamp(utc)  

    #performing sentiment analysis
    ss = sid.polarity_scores(comment)
    
    token_dict = {"comment_body" : comment}
    token_dict.update({"month":dt_object.strftime("%B")})
    token_dict.update({"day" : dt_object.strftime("%d")})
    token_dict.update(ss)
    
    return token_dict

In [5]:
def get_disc_comments(reddit, disc_id):

    num = 0
    sid = SentimentIntensityAnalyzer()
    st = time()
    
    for k in disc_id.keys():
        num += 1
        print("Pulling comments for discussion", num, "of", len(disc_id))
        submission = reddit.submission(k)
        submission.comments.replace_more(limit=0)
        
        for user_comment in submission.comments:
            tok_dict = comment_info(user_comment, submission, sid)
            
            if str(user_comment) not in disc_id[k].keys():
                disc_id[k].setdefault(str(user_comment),tok_dict)
        
    print("\nProcessing time:", round((time()-st)/60, 2), "minutes.")
    return disc_id
        

In [6]:
def add_new_comments(json_file_path, reddit, subreddit):
    
    with open(json_file_path, "r") as f:
        full_comment_dict = json.load(f)
    
        discussion_ids = get_disc_ids(subreddit, full_comment_dict)
        
        all_comment_dict = get_disc_comments(reddit, discussion_ids)
        
        
    #update json file with new comments
    with open(json_file_path, "w") as outfile:
        json.dump(all_comment_dict, outfile, indent = 4)
        
    return all_comment_dict

In [7]:
json_file_path = r"/Users/AllysonEnglish/Active/Active/coronavirus_subreddit_full.json"

In [8]:
# Reddit API wrapper 

reddit = praw.Reddit(client_id=creds.client_id, \
                     client_secret=creds.client_secret, \
                     user_agent=creds.user_agent, \
                     username=creds.username, \
                     password=creds.password)

#Getting daily ID for the Coronavirus daily discussion post

subreddit = reddit.subreddit('Coronavirus')

updated_dict = add_new_comments(json_file_path, reddit, subreddit)

Pulling comments for discussion 1 of 86
Pulling comments for discussion 2 of 86
Pulling comments for discussion 3 of 86
Pulling comments for discussion 4 of 86
Pulling comments for discussion 5 of 86
Pulling comments for discussion 6 of 86
Pulling comments for discussion 7 of 86
Pulling comments for discussion 8 of 86
Pulling comments for discussion 9 of 86
Pulling comments for discussion 10 of 86
Pulling comments for discussion 11 of 86
Pulling comments for discussion 12 of 86
Pulling comments for discussion 13 of 86
Pulling comments for discussion 14 of 86
Pulling comments for discussion 15 of 86
Pulling comments for discussion 16 of 86
Pulling comments for discussion 17 of 86
Pulling comments for discussion 18 of 86
Pulling comments for discussion 19 of 86
Pulling comments for discussion 20 of 86
Pulling comments for discussion 21 of 86
Pulling comments for discussion 22 of 86
Pulling comments for discussion 23 of 86
Pulling comments for discussion 24 of 86
Pulling comments for disc

In [13]:
number = 0
for k in updated_dict.keys():
    for a in updated_dict[k].keys():
        number += len(a)
number

119000

In [12]:
for i, k in enumerate(updated_dict.keys()):
    print(i, k)

0 ghnx5y
1 gh1uuu
2 gftmeo
3 gekees
4 ggfrvq
5 gf6xnu
6 gdbzvl
7 gcqx67
8 gaw27w
9 gdy3p3
10 g8eo6c
11 gbih0c
12 g912p1
13 gc6tdq
14 ga9u7y
15 g61hda
16 fjla5p
17 flad2s
18 g7tugs
19 g6n0i9
20 g9nks9
21 fvz4b2
22 g32elm
23 g5fli1
24 g3nu29
25 ftmp3u
26 fj1m3e
27 fmfa26
28 g47znm
29 g15wnp
30 fsd9v4
31 g2fafj
32 fn0a59
33 g1sbbp
34 fo5geh
35 fpbug6
36 g78qo4
37 flvdic
38 fyg6tm
39 frr6ik
40 fk5m07
41 foqmvt
42 fhgnn3
43 fnkb5o
44 fr66o8
45 fve2wq
46 fiio5o
47 ft068e
48 fz5oik
49 fpxagz
50 fhzyu5
51 fuu28n
52 g0jcbi
53 g4t5ud
54 fwkvfa
55 fu8saj
56 fxssfx
57 fx6u28
58 fzx3wk
59 fqkk56
60 fg9kfi
61 ffqiyn
62 fkpu59
63 fgxhiy
64 fcr8dy
65 gibmed
66 ff8roc
67 fe9i00
68 fcfg0w
69 fevfr5
70 fdr7u4
71 fdcznm
72 fbuxyx
73 fb8ygf
74 fa6etv
75 faowou
76 f9nx69
77 f9az8y
78 f7xqas
79 f8v592
80 f706aq
81 f8n49s
82 f7h02k
83 fere6p
84 fgsu67
85 fgwgiz
